## 임베딩(Embeddings)

이전 예시에서, 길이 `vocab_size`를 가진 고차원 BoW 벡터를 변환하였고, 우리는 명시적으로 저차원 위치 표현 벡터(positional representation vectors)에서 희소 원-핫 표현으로 변환했다. 이러한 원-핫 표현은 메모리 효율이 좋지 않으며, 각 단어는 서로 독립적으로 처리된다. 다시 말해서, 원-핫 인코딩 벡터는 단어 간의 의미적 유사성을 표현하지 않는다.

이 단원에서는 **News AG** 데이터셋을 계속 살펴볼 것이다. 먼저 데이터를 불러오고 이전 유닛에서 몇 가지 정의를 가지고 오겠습니다.


In [1]:
!wget -q https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/torchnlp.py

In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...
Building vocab...
Vocab size =  95812



### 임베딩이란 무엇인가?

**임베딩(embedding)** 의 아이디어는 단어의 의미(semantic meaning)를 반영하는 저차원 밀도 벡터로 단어를 나타내는 것이다. 우리는 나중에 의미 있는 단어 임베딩을 구축하는 방법에 대해 논의할 것이지만, 현재로서는 단어 벡터의 차원 수를 낮추는 방법의 임베딩을 우선 생각해 보자.

따라서 임베딩 레이어는 단어를 입력으로 사용하고 지정된 `embedding_size`의 출력 벡터를 생성한다. 어떤 의미에서는 `Linear` 레이어와 매우 유사하지만, 원-핫 인코딩 벡터를 사용하는 대신 입력으로 단어 숫자를 취할 수 있다.

임베딩 레이어를 네트워크의 첫 번째 레이어로 사용함으로써, 우리는 BoW에서 **embedding bag** 모델로 전환할 수 있으며, 여기서 먼저 텍스트의 각 단어를 해당하는 임베딩으로 변환한 다음, `합계`, `평균` 또는 `최대값`과 같은 모든 임베딩에 대해 집계 함수를 적용할 수 있다.

![5개의 시퀀스 워드에 대한 임베딩 분류기를 보여주는 이미지](./images/embedding-classifier-example.png)

우리의 분류 신경망은 임베딩 레이어, 그 다음 집계 레이어 및 그 위에 선형 분류기로 구성될 것입니다:

In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### 다양한 길이의 시퀀스 다루기

이 아키텍처의 결과로, 네트워크에 대한 미니배치(minibatches)는 특정한 방법으로 생성되어야 합니다. 이전 섹션에서 BoW(bag-of-word)을 사용할 때는 텍스트 시퀀스의 실제 길이에 관계없이 미니배치의 모든 BoW 텐서의 크기가 `vocab_size`와 같았습니다. 일단 단어 임베딩에서 보면 우리는 각 텍스트 샘플에서 다양한 길이의 단어를 얻게 될 것이고, 이러한 샘플을 미니배치로 결합할 때 우리는 패딩(padding)을 적용해야 할 것입니다.

이 작업은 데이터소스에 `collate_fn` 함수를 제공하는 동일한 기술을 사용하여 수행할 수 있습니다:

In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### 임베딩 분류기 학습하기

이제 적절한 데이터로더를 정의했으므로 이전 섹션에서 정의한 훈련 함수를 사용하여 모델을 훈련할 수 있습니다:

In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6428125
6400: acc=0.68453125
9600: acc=0.7123958333333333
12800: acc=0.725703125
16000: acc=0.7365625
19200: acc=0.7464583333333333
22400: acc=0.7548214285714285


(0.9526769402541186, 0.7595969289827256)

> **참조**: 시간을 위해 여기서 25,000개의 레코드(전체 1 epoch 미만)에 대해서만 훈련하고 있지만, 더 높은 정확도를 달성하기 위해 훈련을 계속할 수 있고, 특정 epoch 동안 훈련할 함수를 작성하고, 학습 속도 파라미터를 실험할 수 있다. 정확도는 90% 정도 나타날 것입니다.

### EmbeddingBag Layer and Variable-Length Sequence Representation

이전 아키텍처에서는 미니배치에 맞추기 위해 모든 시퀀스를 동일한 길이로 패딩해야 했습니다. 이것은 가변 길이의 시퀀스를 나타내는 가장 효율적인 방법이 아닙니다. 다른 접근법은 **오프셋(offset)** 벡터를 사용하는 것인데, 이는 하나의 큰 벡터에 저장된 모든 시퀀스의 오프셋을 보유할 것이다.

![오프셋 시퀀스 표현을 보여주는 이미지](./images/offset-sequence-representation.png)

> **참조**: 위 그림에서 일련의 문자를 보여주지만, 다음 예시에서 단어의 시퀀스를 이용합니다. 그러나 오프셋 벡터로 시퀀스를 나타내는 일반적인 원리는 그대로 유지됩니다.

오프셋 표현으로 나타내기 위하여 [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) 레이어를 사용합니다. `Embedding`과 유사하지만 콘텐츠 벡터, 오프셋 벡터를 입력으로 사용하고 `mean`, `sum`, `max` 등 집계 레이어도 포함하고 있다.

다음은 `EmbeddingBag`을 사용하는 수정된 네트워크입니다:

In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

모델 훈련을 위한 데이터셋을 준비하려면 오프셋 벡터를 준비하는 변환 함수를 제공해야 한다:

In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

참고, 이전의 모든 예시와는 달리, 우리의 네트워크는 이제 서로 다른 크기의 데이터 벡터 및 오프셋 벡터라는 두 가지 파라미터를 입력으로 받는다. 마찬가지로, 데이터로더는 2개의 값 대신 3개의 값을 제공합니다. 텍스트 및 오프셋 벡터는 모두 feature로서 제공됩니다. 따라서 다음과 같은 사항을 처리하기 위해 훈련 함수를 약간 조정해야 합니다.

In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6334375
6400: acc=0.68234375
9600: acc=0.7072916666666667
12800: acc=0.72375
16000: acc=0.73575
19200: acc=0.743125
22400: acc=0.7497767857142857


(23.37446267194498, 0.754118682021753)


## Semantic Embeddings: Word2Vec

우리의 이전 예시에서 모델 임베딩 레이어는 단어를 벡터 표현에 매핑하는 방법을 배웠지만, 이 표현은 많은 의미론적 의미(semantical meaning)를 갖지 못했다. 유사한 단어가 벡터 거리(유클리드 거리) 측면에서 서로 가까운 벡터에 해당한다는 벡터 표현을 배우면 좋을 것입니다.

이를 위해, 우리는 특정한 방식으로 대규모 텍스트 콜렉션에 대한 임베딩 모델을 사전 훈련할 필요가 있다. 의미론적 임베딩(Semantic Embeddings)을 훈련시키는 첫 번째 방법 중 하나는 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) 이다. 이것은 단어의 분산 표현을 생성하는 데 사용되는 두 가지 주요 구조에 기초한다:

 - **Continuous bag-of-words** (CBoW) — 이 아키텍처에서, 우리는 주변 문맥에서 단어를 예측하도록 모델을 훈련시킨다. ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ 이 주어지면, 모델의 목표는 $(W_{-2},W_{-1},W_1,W_2)$ 로부터 $W_0$ 를 예측하는 것이다.
 - **Continuous skip-gram** 는 CBoW와 반대이다. 모델은 현재 단어를 예측하기 위해 주변 문맥의 윈도우를 사용합니다.

![단어를 벡터로 변환하기 위한 CBoW 및 Skip-Gram 알고리즘을 모두 보여주는 이미지](./images/example-algorithms-for-converting-words-to-vectors.png)

구글 뉴스 데이터셋에 대하여 사전 훈련된 word2vec 임베딩을 실험하기 위해서는 gensim 라이브러리를 사용한다. 아래에는 'neural'과 가장 유사한 단어가 나와 있습니다.

> **참조:** 단어 벡터를 처음 만들 때, 단어 벡터를 다운로드하는 데 시간이 걸릴 수 있습니다

In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.780479907989502
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851128578186
neuron -> 0.7174385190010071
cortical -> 0.6941086053848267
brain_circuitry -> 0.6923245787620544
synaptic -> 0.6699119210243225
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


또한 분류 모델 훈련에 사용될 단어로부터 임베딩 벡터를 추출할 수 있다(명확성을 위하여 벡터의 처음 20개 컴포넌트만 출력):

In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

의미론적 임베딩(semantical embeddings)의 좋은 점은 인코딩 벡터를 조작하여 의미를 변경할 수 있다는 것입니다. 예를 들어, 우리는 단어를 찾도록 요청할 수 있는데, 그 벡터 표현은 가능한 한 단어 *king* 과 *woman*와 가깝고, 그리고 *man*이라는 단어에서 멀리 떨어져 있습니다.

In [11]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBOW와 Skip-Grams는 모두 로컬 컨텍스트를 고려한다는 점에서 "예측적(predictive)" 임베딩이다. Word2Vec은 글로벌 컨텍스트를 이용하지 않는다.

**FastText** 는 각 단어에 대한 벡터 표현과 n-grams 문자에 대해 학습함으로써 Word2Vec을 구축한다. 그런 다음 벡터 표현의 값은 각 훈련 스텝에서 하나의 벡터로 평균 처리된다. 이것은 사전 훈련에 많은 계산을 추가하지만, 단어 임베딩이 하위 단어 정보를 인코딩할 수 있게 한다.

또 다른 방법인 **GloVe**는 co-occurence matrix의 개념을 활용하고 신경 방법(neural methods)을 사용하여 co-occurence matrix을 보다 표현적이고 비선형적인 단어 벡터로 분해한다.

gensim 라이브러리가 지원하므로 임베딩을 FastText 및 GloVe로 변경하여 예제를 실행할 수 있습니다:

## PyTorch에서 사전 학습된 임베딩 사용하기

위의 예시를 수정하여 Word2Vec와 같은 의미론적 임베딩으로 임베딩 레이어의 매트릭스를 미리 채울 수 있다. 사전 훈련된 임베딩과 텍스트 말뭉치의 사전이 일치하지 않을 가능성이 높기 때문에 랜덤 값으로 누락된 단어에 대한 가중치를 초기화한다는 점을 고려해야 한다.

In [12]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.itos):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


이제 우리의 모델을 훈련시키자. 모델을 훈련하는 데 걸리는 시간은 임베딩 레이어 크기가 더 커져서 파라미터의 수가 훨씬 많아지기 때문에 이전 예시보다 훨씬 더 오래 걸리는 점에 유의한다. 또한 이 때문에 과적합을 피하고 싶다면, 더 많은 예시에 대해 우리의 모델을 훈련시킬 필요가 있다.

In [13]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.63875
6400: acc=0.693125
9600: acc=0.7176041666666667
12800: acc=0.7321875
16000: acc=0.7454375
19200: acc=0.7559375
22400: acc=0.7631696428571428


(218.64081493921944, 0.7667146513115803)

아래의 경우 정확도가 크게 증가하지 않으며, 이는 사전이 상당히 다를 수 있다.
서로 다른 사전의 문제를 극복하기 위해 다음 솔루션 중 하나를 사용할 수 있습니다:
* wore2vec 모델을 우리의 사전에 대하여 재학습
* 사전 훈련된 word2vec 모델의 사전으로 데이터셋을 로드한다. 데이터셋을 로드하는 데 사용되는 사전을 로드하는 동안 지정할 수 있습니다.

특히 PyTorch `torchtext` 프레임워크에 임베딩 지원이 내장되어 있기 때문에 후자의 접근 방식이 더 쉬워 보인다. 예를 들어 GloVe 기반 어휘는 다음과 같은 방법으로 인스턴스화할 수 있습니다:

In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

불러온 사전의 기본 연산은 다음과 같습니다:
* `vocab.stoi` 사전은 우리가 단어를 그것의 사전 인덱스로 변환하는 것을 허용
* `vocab.itos` 는 그 반대이다. 숫자를 단어로 변환한다.
* `vocab.vectors` 는 임베딩 벡터의 배열이므로 단어의 임베딩 `s`를 얻으려면 `vocab.vectors[vocab.stoi[s]]`를 사용할 필요가 있다.

다음은 **kind-man+woman = queen** 방정식을 나타내기 위해 임베딩을 조작하는 예입니다

In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

이러한 임베딩을 사용하여 분류기를 훈련하려면, 먼저 GloVe 사전을 사용하여 데이터셋을 인코딩해야 한다:

In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

위에서 살펴본 것처럼, `vocab.vectors`에 저장된 모든 임베딩 벡터는 사전에 저장됩니다. 이를 통해 다음과 같은 간단한 복사를 사용하여 임베딩 레이어의 가중치에 로드하기가 매우 쉽다.

In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

이제 모델을 훈련하고 더 나은 결과를 얻을 수 있는지 알아보겠습니다:

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

정확도가 크게 증가하지 않는 이유 중 하나는 사전 훈련된 GloVe 사전에서 데이터셋의 일부 단어가 누락되어 기본적으로 무시되기 때문이다. 이를 극복하기 위해, 우리는 데이터셋 대한 자체 임베딩을 훈련할 수 있다.


## 자체 임베딩 학습하기

우리의 예시에서, 사전 훈련된 의미 임베딩을 사용해 왔지만, 이러한 임베딩이 CBoW 또는 Skip-gram 아키텍처를 사용하여 어떻게 훈련될 수 있는지 보는 것은 흥미롭다. 이 연습은 이 모듈을 넘어갔지만, 관심 있는 사람들은 이 [official PyTorch tutorial on Language Modeling](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)을 확인할 수 있다. 또한, **gensim** 프레임워크는 [in this documentation](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) 설명한 것처럼 가장 일반적으로 사용되는 몇 줄의 코드에서 임베딩을 훈련시키는 데 사용될 수 있다.

## Contextual Embeddings

Word2Vec와 같은 기성 사전 훈련된 임베딩의 대표적인 한 가지 주요 한계점은 단어 감지 불능화(word sense disambigioution) 문제이다. 사전 훈련된 임베딩은 문맥에서 단어의 의미 일부를 캡처할 수 있지만, 가능한 단어의 모든 의미는 동일한 임베딩으로 인코딩된다. 이는 다운스트림 모델에 문제를 일으킬 수 있는데, 이는 'play'라는 단어와 같이 많은 단어들이 사용되는 문맥에 따라 다른 의미를 가지기 때문이다.

예를 들어, 이 두 문장의 'play'라는 단어는 상당히 다른 의미를 가지고 있다:
- I went to a **play** at the theature.
- John wants to **play** with his friends.

위의 사전 훈련된 임베딩은 동일한 임베딩에서 'play'이라는 단어의 이러한 의미를 모두 나타낸다. 이러한 한계점을 극복하기 위해, 우리는 큰 텍스트 말뭉치에 대해 훈련된 **language model**을 기반으로 임베딩을 구축해야 하며, 다른 컨텍스트에서 단어를 어떻게 조합할 수 있는지 알고 있다. 컨텍스트 임베딩에 대한 논의는 이 튜토리얼의 범위를 벗어났지만 다음 섹션에서 언어 모델에 대해 이야기할 때 다시 살펴보기로 한다.